In [2]:
import wordcloud
import pandas as pd


In [3]:
df = pd.read_csv("../data/aapl_us_equities_news_processed.csv")
df

,text,target
0,UPDATE 4 Rouble at record low as Russia faces ...,1
1,UPDATE 4 DSG to cut more costs as sales fall w...,1
2,Lack of fame aids Geithner over US tax mistake...,1
3,CORRECTED Depression ahead prepare for stoc...,1
4,Euro area CPI Falls Down To 1 6 In December r...,1
...,...,...
196854,Dow Jones News Cisco Reports Today Disney Bo...,1
196855,Cisco beats on earnings shares barely budge c...,1
196856,Cisco Systems CSCO Q2 Earnings And Revenues ...,1
196857,Why Teva Pharmaceutical Is Rallying 10 Today ...,1
